In [1]:
# This Python 3 environment comes with ma
# ny helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Install required libraries for this notebook
!pip install -U google-generativeai python-dotenv pymupdf



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# TaskFlow: An Autonomous Research & Execution Agent 🧠⚙️

This project demonstrates a GenAI-based autonomous agent that:
- Understands complex research queries
- Breaks them into subtasks
- Retrieves and summarizes relevant info using LLM
- Outputs structured research reports (in JSON/Markdown)

Models used: `gemini-1.5-pro`, `text-embedding-004`


In [3]:
import google.generativeai as genai
import json

# Show the installed google-generativeai version
genai.__version__


'0.8.5'

In [4]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

for m in genai.list_models():
    print(m.name)


models/embedding-gecko-001
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image-preview
models/gemini-2.5-flash-image
models/gemini-2.5-flash-preview-09-2025
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robo

In [5]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content("Hello! This is a test.")
print(response.text)


Hello! I'm ready. How can I help you today, or what would you like to test?


In [6]:
from google.api_core import retry

def is_retriable(exception):
    return isinstance(exception, Exception) and (
        "429" in str(exception) or "503" in str(exception)
    )

retry_wrapper = retry.Retry(predicate=is_retriable)

def generate_with_retry(model, *args, **kwargs):
    return retry_wrapper(model.generate_content)(*args, **kwargs)


### Using the gemini-1.5-pro, and answering a prompt 

In [7]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

short_config = {"max_output_tokens": 2000}

model = genai.GenerativeModel("gemini-2.5-flash")

response = model.generate_content(
    "Find best open-source LLMs with license info in 200 words",
    generation_config=short_config,
)

print(response.text)


Finding the "best" open-source LLMs depends on your specific needs, but the leaders balance performance with permissive licensing. While some are truly open-source (OSI-approved), others are "openly available" with commercial terms. Always consult the specific license.

1.  **Llama 3 (Meta):** The latest iteration (8B, 70B models) from Meta offers state-of-the-art performance and a vast ecosystem. Its **Llama 3 Community License** is significantly more permissive than Llama 2's, allowing commercial use without the previous monthly active user (MAU) cap. This makes it a top choice for broad applications.

2.  **Mistral AI Models (e.g., Mistral 7B, Mixtral 8x7B):** Mistral AI provides incredibly efficient and powerful models. Mistral 7B is an excellent small model, and Mixtral 8x7B (a sparse Mixture of Experts) delivers exceptional performance. Crucially, these foundational models are released under the highly permissive **Apache 2.0 license**, making them truly open-source and ideal for

### Responding some user queries and returning the response in json format

In [8]:
import google.generativeai as genai
import typing_extensions as typing
import json
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

class TaskPlan(typing.TypedDict):
    tasks: list[str]

generation_config = {
    "temperature": 0.2,
    "response_mime_type": "application/json",
    "response_schema": TaskPlan,
}

model = genai.GenerativeModel("gemini-2.5-flash")

def plan_tasks(topic: str):
    response = model.generate_content(
        f"Break into 3–5 research tasks about: '{topic}'. "
        "Return JSON with a single key 'tasks' containing an array of task strings.",
        generation_config=generation_config,
    )
    return response.text

user_query = "Role of AI in sustainable development"  # queries
tasks = plan_tasks(user_query)

parsed_tasks = json.loads(tasks)
print(json.dumps(parsed_tasks, indent=2))


{
  "tasks": [
    "Identify current applications of AI in various sectors of sustainable development (e.g., energy, agriculture, smart cities).",
    "Analyze the potential benefits and contributions of AI towards achieving specific Sustainable Development Goals (SDGs).",
    "Examine the challenges, risks, and ethical considerations associated with AI deployment in sustainable development initiatives.",
    "Investigate policy frameworks, governance models, and international collaborations needed to ensure responsible and equitable AI use for sustainability.",
    "Explore future trends and emerging AI technologies that could further accelerate progress in sustainable development."
  ]
}


# Agent Loop
[ User query ] → [ Task planner ] → [ Subtasks ] → [ Execute each task ]

- It decomposes each tasks into subtasks
- Then executes each subtasks using GenAI


In [9]:
import google.generativeai as genai
model = genai.GenerativeModel(model_name="gemini-2.5-flash") # using a different model as gemini 1.5 pro has limitations on rpm



for i, task in enumerate(parsed_tasks['tasks'], 1):
    print(f"\n🔹 Task {i}: {task}")
    
    # Ask Gemini to answer each task
    response = model.generate_content(f"Write a short paragraph on: {task}")
    
    print("📝 Response:")
    print(response.text)



🔹 Task 1: Identify current applications of AI in various sectors of sustainable development (e.g., energy, agriculture, smart cities).
📝 Response:
Artificial intelligence is increasingly pivotal in driving sustainable development across diverse sectors. In the **energy sector**, AI optimizes smart grids by predicting demand and supply fluctuations, efficiently integrating renewable sources like solar and wind power, and enhancing energy conservation in buildings through predictive control systems. For **agriculture**, AI-driven precision farming minimizes resource waste; it enables intelligent irrigation, targeted fertilization based on soil and crop health data, and early detection of diseases or pests through image recognition from drones and satellites. Within **smart cities**, AI applications range from optimizing traffic flow and public transport routes to reduce congestion and emissions, to intelligent waste management systems that predict collection needs and optimize routes, t

# Document Understanding

Reading documents and extracting text from it

In [10]:
!pip install pymupdf

import fitz  # PyMuPDF

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

raw_text = extract_text_from_pdf('H:\GATE SYLLABUS.pdf')


<>:12: SyntaxWarning: invalid escape sequence '\G'
<>:12: SyntaxWarning: invalid escape sequence '\G'
C:\Users\hp\AppData\Local\Temp\ipykernel_22608\1839195758.py:12: SyntaxWarning: invalid escape sequence '\G'
  raw_text = extract_text_from_pdf('H:\GATE SYLLABUS.pdf')



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Breaking the long pdf into chunks so that it is made easier for the model to understand and give output efficiently with minimal errors.

In [11]:
def chunk_text(text, chunk_size=100, overlap=50):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks

chunks = chunk_text(raw_text)

### Embed the chunks

Embedding the chunks into vectors for semantic understanding.

using embedding model for embedding the chunks.

In [12]:
# Batching the chunks as the request is limited to 100
import google.generativeai as genai
import numpy as np

def batch_chunks(chunks, batch_size=100):
    for i in range(0, len(chunks), batch_size):
        yield chunks[i:i + batch_size]

all_embeddings = []

for batch in batch_chunks(chunks, batch_size=100):
    response = genai.embed_content(
        model="text-embedding-004",
        content=batch,
        task_type="semantic_similarity",
    )

    # Case 1: batch of many → response["embeddings"]
    if "embeddings" in response:
        vectors = [item["embedding"] for item in response["embeddings"]]

    # Case 2: batch of one → response["embedding"]
    elif "embedding" in response:
        vectors = [response["embedding"]]

    else:
        raise ValueError("Unexpected embedding response format:", response)

    # FIX: Convert token-level embeddings (N,768) to sentence-level (768,)
    processed_vectors = []
    for v in vectors:
        v = np.array(v)
        if v.ndim == 2:            # shape (N, 768)
            v = v.mean(axis=0)     # convert to (768,)
        processed_vectors.append(v.tolist())

    all_embeddings.extend(processed_vectors)


The embeddedd vectors are now being stored in chunk_db

In [13]:
# Assuming all_embeddings and chunks are in order
chunk_db = [
    {"text": chunk, "embedding": embedding}
    for chunk, embedding in zip(chunks, all_embeddings)
]


Also embedding the query into vectors so that the model can match these embedding with the relevant content from the pdf.

In [14]:
query = "what is the doucment about"

In [15]:
import google.generativeai as genai

query_embedding_response = genai.embed_content(
    model="text-embedding-004",
    content=query,
    task_type="semantic_similarity",
)

# Single embedding vector for the query
query_embedding = query_embedding_response.get("embedding") or query_embedding_response.get("embeddings", [None])[0]


Calculating the similarity scores using cosine similarity and finding the top k similar chunks

In [16]:
from numpy import dot
from numpy.linalg import norm
import numpy as np

def cosine_similarity(a, b):
    return dot(a, b) / (norm(a) * norm(b))

top_k = 3

scored_chunks = [
    (cosine_similarity(np.array(query_embedding), np.array(chunk["embedding"])), chunk["text"])
    for chunk in chunk_db
]

ranked_chunks = sorted(scored_chunks, key=lambda x: x[0], reverse=True)
top_chunks = [text for _, text in ranked_chunks[:top_k]]


Giving a rag prompt that specifies what to do to the model

and generating the output using the gemini model.


In [17]:
print(type(chunks[0]))
print(chunks[0][:200])


<class 'str'>
 
CS Computer Science and Information Technology 
Section 1: Engineering Mathematics 
Discrete Mathe


In [18]:
rag_prompt = (
    "Answer the following question using the provided context.\n\n"
    f"Context:\n{''.join(top_chunks)}\n\n"
    f"Question:\n{query}"
)

import google.generativeai as genai

response = genai.GenerativeModel("gemini-2.5-flash").generate_content(rag_prompt)

print("🔍 Final Answer:\n", response.text)


🔍 Final Answer:
 Based on the provided context, the document is about **Discrete Mathematics**, which falls under **Engineering Mathematics** within the broader field of **Computer Science and Information Technology**.
